### Code to create test files for vdj_asm unit testing.

In [34]:
import tenkit.bam as tk_bam
import tenkit.seq as tk_seq
import numpy as np
import numpy.random as random
import pyfasta
import pysam
import os
import os.path
import re

NUCS = ['A', 'C', 'G', 'T']

UNMAPPED_FIRST_FLAG = int('0b1001101', 2)
UNMAPPED_SECOND_FLAG = int('0b10001101', 2)
MAPPED_FIRST_FLAG = int('0b1100011', 2)
MAPPED_SECOND_FLAG = int('0b10010011', 2)

MAPPED_UNPAIRED_FLAG = int('0b1000000', 2)
UNMAPPED_UNPAIRED_FLAG = int('0b1000100', 2)

### Create a small bam for testing indexing and bam merging in vdj_asm

In [40]:
in_bam_name = '/mnt/park/gemcode/sofia/vdj/master_runs/pipestances/31466/SC_VDJ_PD/SC_VDJ/_VDJ_ASSEMBLER/FILTER_VDJ_READS/fork0/chnk0/files/AAACGGGTCGTTTATC-1.bam'
outdir = '/mnt/home/sofia/yard/vdj/data/vdj_asm_test/'

out_bam_name = os.path.join(outdir, 'test_index.bam')

split_out_bam_names = [os.path.join(outdir, 'test_split1.bam'),
                       os.path.join(outdir, 'test_split2.bam')]

chroms = ['chr1', 'chr2', 'chr3', 'chr4', 'chr5']
lengths = [1000, 1000, 1000, 1000, 1000]

# Split chromosomes across two bams. One of the chromosomes will span both bams.
split_chroms = [[True, True, True, False, False],
                [False, False, True, True, True]]

readpairs_per_chrom = 3 # the last pair will always be unmapped

# Order of chromosomes in the output file (we want the output to be non-sorted).
# The output will have #chroms blocks, each block containing readpairs_per_chrom pairs.
chrom_order = np.array([1, 0, 2, 4, 3])

in_bam = tk_bam.create_bam_infile(in_bam_name)
out_bam, tids = tk_bam.create_bam_outfile(out_bam_name, chroms, lengths)
print 'tids', tids

split_out_bams = [tk_bam.create_bam_outfile(n,
                                            [chroms[i] for i in range(len(sc)) if sc[i]],
                                            [lengths[i] for i in range(len(sc)) if sc[i]])
                  for (n, sc) in zip(split_out_bam_names, split_chroms)]

print 'Split tids', [t for (c, t) in split_out_bams]

output_on_common = False

for ridx, read in enumerate(in_bam):
    if ridx == 2 * readpairs_per_chrom * len(lengths):
        break
        
    if ridx % (2 * readpairs_per_chrom) < 2 * readpairs_per_chrom - 2:
        # This is not the last pair of the block.
        read.reference_id = chrom_order[ridx / (2 * readpairs_per_chrom)]
        read.reference_start = 100 * (ridx % 2)
        read.next_reference_id = read.tid
        read.next_reference_start = 100 * ((ridx + 1) % 2)
        read.flag = MAPPED_FIRST_FLAG if ridx % 2 == 0 else MAPPED_SECOND_FLAG
        
        assert(not read.is_unmapped)
        assert(not read.mate_is_unmapped)
        assert(not read.is_secondary)
        assert(read.is_paired)
        assert((read.is_reverse and ridx % 2 == 1) or (not read.is_reverse and ridx % 2 == 0))
    else: 
        read.reference_id = -1
        read.reference_start = 0
        read.flag = UNMAPPED_FIRST_FLAG if ridx % 2 == 0 else UNMAPPED_SECOND_FLAG
        read.next_reference_id = -1
        read.next_reference_start = 0
        assert(read.is_paired)
        assert(read.is_unmapped)
        assert(read.mate_is_unmapped)
        
    read.qname = 'read{}|||CB|||AAACGGGT|||UB|||ACGT'.format(ridx / 2)
    
    out_bam.write(read)
    
    # Decide on which split this read should go
    # If the chromosome of the read belongs to the first split, put it there
    # unless this is the chromosome that spans both splits.
    if split_chroms[0][chrom_order[ridx / (2 * readpairs_per_chrom)]]:
        if split_chroms[1][chrom_order[ridx / (2 * readpairs_per_chrom)]]:
            if output_on_common:
                split_bam = 1
            else:
                split_bam = 0
                # We ouput a pair on the common chromosome. All other pairs 
                # on that chromosome will go to the second split.
                if ridx % 2 == 1:
                    output_on_common = True
        else:
            split_bam = 0
    else:
        split_bam = 1
        
    # tids are different!
    if read.reference_id > -1:
        read.reference_id = split_out_bams[split_bam][1][out_bam.references[read.reference_id]]
        read.next_reference_id = read.reference_id
        
    split_out_bams[split_bam][0].write(read)
    
out_bam.close()

for split_bam in split_out_bams:
    split_bam[0].close() 
    
tk_bam.sort_and_index(out_bam_name, re.sub('.bam', '_sorted', out_bam_name))

tids {'chr5': 4, 'chr4': 3, 'chr3': 2, 'chr2': 1, 'chr1': 0}
Split tids [{'chr3': 2, 'chr2': 1, 'chr1': 0}, {'chr5': 2, 'chr4': 1, 'chr3': 0}]


### Test bam for assembly

In [36]:
out_bam_name = '/mnt/home/sofia/yard/vdj/data/vdj_asm_test/test_asm.bam'

def unmapped_read(first=True):
    read = pysam.AlignedRead()
    read.flag = UNMAPPED_FIRST_FLAG if first else UNMAPPED_SECOND_FLAG
    read.reference_id = -1
    read.reference_start = 0
    read.next_reference_id = -1
    read.next_reference_start = 0
    return read

READ_LEN = 150

# chromosomes and positions of reads don't matter
out_bam, _ = tk_bam.create_bam_outfile(out_bam_name, ['chr1'], [1000])

gt_fasta = pyfasta.Fasta('/mnt/opt/meowmix_git/cellranger/vdj/ground_truths/jurkat/contigs.fasta')

# We'll create one UMI for each barcode and ground truth sequence combination
# For each UMI, we'll get pairs_per_bc read pairs. These will be sequences from
# the ground truth, spaced at a given step. So we should be able to reconstruct
# the first ((p - 1) * 20) + READ_LEN bases of each ground truth sequence,
# where p is the number of pairs.
# In addition to the pairs above, we'll also create an unmapped pair which should 
# be ignored by the assembler.
barcodes = ['AAACGGGT', 'CAACGGGT', 'GAACGGGT']
umis = ['ACGT', 'ACCC']

pairs_per_bc = [5, 4, 3]
# Total number of pairs (+ 3 because of one unmapped per BC):
# 2 * (5 + 4 + 3) + 3 = 27 (or 54 reads)

step = 40

for bc_idx, bc in enumerate(barcodes):
    # Create one unmapped pair
    read1 = unmapped_read(True)
    read2 = unmapped_read(False)
    read1.seq = 'A' * READ_LEN
    read2.seq = read1.seq
    read1.qual = chr(63) * len(read1.seq)
    read2.qual = chr(63) * len(read2.seq)
    
    qname = 'read{}:{}|||SEQ|||{}|||CB|||{}|||UB|||AAAA'.format(0, bc, 'FOO', bc)
    read1.qname = qname 
    read2.qname = qname

    out_bam.write(read1)
    out_bam.write(read2)
            
    for ridx in range(pairs_per_bc[bc_idx]):
        for (seq_name, seq), umi in zip(gt_fasta.iteritems(), umis):
            
            read1 = pysam.AlignedRead()
            read2 = pysam.AlignedRead()
            
            read1.reference_id = 0
            read1.reference_start = 0
            read2.reference_id = 0
            read2.reference_start = 0
            read1.next_reference_id = 0
            read1.next_reference_start = 0
            read2.next_reference_id = 0
            read2.next_reference_start = 0
    
            read1.flag = MAPPED_FIRST_FLAG
            read2.flag = MAPPED_SECOND_FLAG
            
            read1.seq = str(seq[0:][(step * ridx):(step * ridx + READ_LEN)].upper())
            read2.seq = read1.seq
            
            read1.qual = chr(63) * len(read1.seq)
            read2.qual = chr(63) * len(read2.seq)
            
            qname = 'read{}:{}|||SEQ|||{}|||CB|||{}|||UB|||{}'.format(ridx + 1, bc, seq_name, bc, umi)
            read1.qname = qname 
            read2.qname = qname
            
            out_bam.write(read1)
            out_bam.write(read2)
        

out_bam.close()

# Write the sequences that we expect to reconstruct
with open('/mnt/home/sofia/yard/vdj/data/vdj_asm_test/test_asm.fasta', 'w') as out_fasta:
    for bc_idx, bc in enumerate(barcodes):
        for (seq_name, seq), umi in zip(gt_fasta.iteritems(), umis):
            s = seq[0:((pairs_per_bc[bc_idx] - 1) * step + READ_LEN)].upper()
            out_fasta.write('>{}_{}\n'.format(bc, umi))
            out_fasta.write(s + '\n')

### Test single-end bam for assembly

In [37]:
out_bam_name = '/mnt/home/sofia/yard/vdj/data/vdj_asm_test/test_asm_se.bam'

def unmapped_read():
    read = pysam.AlignedRead()
    read.flag = UNMAPPED_UNPAIRED_FLAG
    read.reference_id = -1
    read.reference_start = 0
    return read

READ_LEN = 150

# chromosomes and positions of reads don't matter
out_bam, _ = tk_bam.create_bam_outfile(out_bam_name, ['chr1'], [1000])

gt_fasta = pyfasta.Fasta('/mnt/opt/meowmix_git/cellranger/vdj/ground_truths/jurkat/contigs.fasta')

# We'll create one UMI for each barcode and ground truth sequence combination
# For each UMI, we'll get reads_per_bc readss. These will be sequences from
# the ground truth, spaced at a given step. So we should be able to reconstruct
# the first ((p - 1) * 20) + READ_LEN bases of each ground truth sequence,
# where p is the number of reads.
# I.e. we should reconstruct the same sequences as in the paired case (if we use all 
# reads).
# In addition to the reads above, we'll also create an unmapped read which should 
# be ignored by the assembler. These unmapped reads will be interleaved with the 
# mapped ones.
barcodes = ['AAACGGGT', 'CAACGGGT', 'GAACGGGT']
umis = ['ACGT', 'ACCC']

reads_per_bc = [5, 4, 3]
# Total number of reads (* 2 because of the unmapped reads and then * 2 by chain):
# 2 * 2 * (5 + 4 + 3) = 48

step = 40

for bc_idx, bc in enumerate(barcodes):
    nreads = 0        
    for ridx in range(reads_per_bc[bc_idx]):
        for (seq_name, seq), umi in zip(gt_fasta.iteritems(), umis):
            
            read1 = pysam.AlignedRead()
            
            read1.reference_id = 0
            read1.reference_start = 0
    
            read1.flag = MAPPED_UNPAIRED_FLAG
            
            read1.seq = str(seq[0:][(step * ridx):(step * ridx + READ_LEN)].upper())
            read1.qual = chr(63) * len(read1.seq)
            
            qname = 'read{}:{}|||SEQ|||{}|||CB|||{}|||UB|||{}'.format(nreads, bc, seq_name, bc, umi)
            read1.qname = qname 
            
            # Create one unmapped read
            read2 = unmapped_read()
            read2.seq = 'A' * READ_LEN
            read2.qual = chr(63) * len(read2.seq)

            qname = 'read{}:{}|||SEQ|||{}|||CB|||{}|||UB|||AAAA'.format(nreads + 1, bc, seq_name, bc)
            read2.qname = qname

            out_bam.write(read1)
            out_bam.write(read2)
            nreads += 2
            
out_bam.close()

### Create input data for testing the base quality computation

In [38]:
import numpy as np
import numpy.random
import tenkit.fasta as tk_fasta
import tenkit.seq as tk_seq

def create_random_seq(seq_len, letters = NUCS):
    output_seq_letters = np.random.choice(np.arange(4), seq_len, replace=True)
    return ''.join([letters[i] for i in output_seq_letters])

def write_reads(gt_seq, start, read_len, umi, qidx, fq1, fq2, fq, error_rate=0):
    seq1 = gt_seq[start:(start + read_len)]
    seq2 = gt_seq[(start + read_len):(start + 2 * read_len)] 
    
    if error_rate > 0:
        seq1 = ''.join([np.random.choice(NUCS, 1)[0] if np.random.rand() < error_rate else s for s in seq1])
        seq2 = ''.join([np.random.choice(NUCS, 1)[0] if np.random.rand() < error_rate else s for s in seq2])
        
    quals = chr(60) * len(seq1)
    
    bc = 'ACGTACGT'
    qname = 'read{}|||CB|||{}|||UB|||{}'.format(qidx, bc, umi)
    tk_fasta.write_read_fastq(fq1, qname, seq1, quals)
    tk_fasta.write_read_fastq(fq2, qname, tk_seq.get_rev_comp(seq2), quals)
    
    tk_fasta.write_read_fastq(fq, qname, seq1, quals)
    qname = 'read{}b|||CB|||{}|||UB|||{}'.format(qidx, bc, umi)
    tk_fasta.write_read_fastq(fq, qname, seq2, quals)
    
np.random.seed(1)

output_seq_len = 1000
output_seq = create_random_seq(output_seq_len)

outdir = '/mnt/home/sofia/yard/vdj/data/vdj_asm_test/base_quals/'

if not os.path.isdir(outdir):
    os.mkdir(outdir)
    
out_fasta_name = os.path.join(outdir, 'test_base_quals.fasta')
out_fq1_name = os.path.join(outdir, 'test_base_quals_1.fastq')
out_fq2_name = os.path.join(outdir, 'test_base_quals_2.fastq')
out_fq_name = os.path.join(outdir, 'test_base_quals.fastq')

with open(out_fasta_name, 'w') as f:
    tk_fasta.write_read_fasta(f, 'random_seq', output_seq)
    
read_len = 100

with open(out_fq1_name, 'w') as fq1, open(out_fq2_name, 'w') as fq2, open(out_fq_name, 'w') as fq:
    write_reads(output_seq, 0, read_len, 'ACGT', 0, fq1, fq2, fq)
    write_reads(output_seq, 200, read_len, 'ACGT', 1, fq1, fq2, fq)
    write_reads(output_seq, 0, read_len, 'CCCC', 2, fq1, fq2, fq)
    
    write_reads(output_seq, 400, read_len, 'CCCC', 3, fq1, fq2, fq, 0.5)

In [39]:
seq1 = 'ACGTA'
[np.random.choice(letters, 1)[0] if np.random.rand() < 0.1 else s for s in seq1]

['A', 'C', 'G', 'T', 'A']